In [ ]:
import cv2
import numpy as np 
import pandas as pd 
import os
from PIL import Image
from tensorflow.keras.applications.resnet50 import ResNet50
import tensorflow as tf
import tensorflow.keras as K
from tensorflow.keras import datasets, layers, models, losses, Model
from sklearn.model_selection import train_test_split
from sklearn import preprocessing as pre
from glob import glob
from sklearn.utils.class_weight import compute_class_weight
import imgaug as ia
import imgaug.augmenters as iaa
os.environ["CUDA_VISIBLE_DEVICES"] = '2,3'


In [ ]:
Train_img_list=pd.read_csv('../../data/standardFrame_data/scale_skip/Train_dataframe.csv')['file_path'].to_list()
Train_label_list=pd.read_csv('../../data/standardFrame_data/scale_skip/Train_dataframe.csv')['standard'].to_list()
Test_img_list=pd.read_csv('../../data/standardFrame_data/scale_skip/Test_dataframe.csv')['file_path'].to_list()
Test_label_list=pd.read_csv('../../data/standardFrame_data/scale_skip/Test_dataframe.csv')['standard'].to_list()
Val_img_list=pd.read_csv('../../data/standardFrame_data/scale_skip/Validation_dataframe.csv')['file_path'].to_list()
Val_label_list=pd.read_csv('../../data/standardFrame_data/scale_skip/Validation_dataframe.csv')['standard'].to_list()

In [ ]:
Train_img_path='../../data/standardFrame_data/scale_skip/train'
Test_img_path='../../data/standardFrame_data/scale_skip/test'
Val_img_path='../../data/standardFrame_data/scale_skip/val'
seq = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.Flipud(0.5),
    iaa.Affine(
            translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
            rotate=(-30, 30),
            shear=(-16, 16)
            ),
    iaa.TranslateX(px=(-20, 20)),
    iaa.TranslateY(px=(-20, 20)),

], random_order=True)

In [ ]:
size=224
x_train = np.zeros((len(Train_img_list)*4,size,size,3))
y_train = np.zeros((len(Train_img_list)*4))
for i in range(len(Train_img_list)):
    x_train[i*4] =cv2.cvtColor(np.array(Image.open(Train_img_path+Train_img_list[i]).resize((size,size)).convert('L')),cv2.COLOR_GRAY2RGB)
    x_train[i*4+1] =cv2.cvtColor(seq(images=np.array(Image.open(Train_img_path+Train_img_list[i]).resize((size,size)).convert('L'))),cv2.COLOR_GRAY2RGB)
    x_train[i*4+2] =cv2.cvtColor(seq(images=np.array(Image.open(Train_img_path+Train_img_list[i]).resize((size,size)).convert('L'))),cv2.COLOR_GRAY2RGB)
    x_train[i*4+3] =cv2.cvtColor(seq(images=np.array(Image.open(Train_img_path+Train_img_list[i]).resize((size,size)).convert('L'))),cv2.COLOR_GRAY2RGB)
    y_train[i*4]=Train_label_list[i]
    y_train[i*4+1]=Train_label_list[i]
    y_train[i*4+2]=Train_label_list[i]
    y_train[i*4+3]=Train_label_list[i]
    
x_train=x_train/255

x_test = np.zeros((len(Test_img_list),size,size,3))
for i in range(len(Test_img_list)):
    x_test[i] =np.array(Image.open(Test_img_path+Test_img_list[i]).resize((size,size)))
x_test=x_test/255
y_test=np.array(Test_label_list)

x_val = np.zeros((len(Val_img_list),size,size,3))
for i in range(len(Val_img_list)):
    x_val[i] =np.array(Image.open(Val_img_path+Val_img_list[i]).resize((size,size)))
x_val=x_val/255
y_val=np.array(Val_label_list)

In [ ]:
def batch_generator(image, label, batchsize):
    N = len(image)
    indices = np.arange(N)  # 0부터 N-1까지의 인덱스 배열 생성
    np.random.shuffle(indices)  # 인덱스 배열을 무작위로 섞음

    i = 0
    while True:
        if i + batchsize <= N:
            batch_indices = indices[i:i+batchsize]
            i = i + batchsize
        else:  # 남은 데이터가 batchsize보다 작을 때, 배열을 wrap around하여 다시 섞음
            batch_indices = np.concatenate((indices[i:], indices[:batchsize - (N - i)]))
            i = batchsize - (N - i)

            np.random.shuffle(indices)  # 다음 에포크를 위해 인덱스 배열을 무작위로 섞음

        yield image[batch_indices], label[batch_indices]

In [ ]:

checkpoint_filepath = "../../model/skip/MobileNetV2_ada_checkpoints.h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only= True
)
class_weight_ratio=compute_class_weight(class_weight = "balanced" , 
                     classes=np.unique(y_train), 
                     y = y_train)
class_weight = {0:class_weight_ratio[0],1:class_weight_ratio[1]}
mirrored_strategy = tf.distribute.MirroredStrategy(devices=['/GPU:0','/GPU:1'])
with mirrored_strategy.scope(): 
    input_t=K.Input(shape=(size,size, 3))
    input_tensor = layers.experimental.preprocessing.Resizing(size, size, interpolation="bilinear", input_shape=x_train.shape[1:])(input_t)
    ResNet=K.applications.MobileNetV2(include_top=True,weights='imagenet',input_tensor=input_tensor)
    model = K.models.Sequential()
    model.add(ResNet)
    model.add(tf.keras.layers.Dropout(.2, input_shape=(64,)))
    model.add(K.layers.Dense(64, activation=tf.keras.layers.LeakyReLU(alpha=0.1)))
    model.add(K.layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer=K.optimizers.Adam(lr=2e-4),
                    loss=tf.keras.losses.binary_crossentropy,
                    metrics=["accuracy"])
    histo=model.fit(
        batch_generator(x_train,y_train,64),
        validation_data=(x_val,y_val),
        epochs=500,
        steps_per_epoch=len(x_train)//64,
        callbacks=[model_checkpoint_callback],
        shuffle=True,
        class_weight=class_weight
    )
    model.save('../../model/skip/MobileNetV2_ada.h5')